# $H_2O$ cooling rate profiles

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [45]:
import os
import sys
import shutil
import pprint
from pathlib import *

import pymongo
import json
import pandas as pd
import xarray as xr

from climatools.lblnew.bestfit_params import *
from climatools.atm import *
from climatools.parameters import *
from climatools.dataset import *
from climatools.spectrum import *

In [12]:
client = pymongo.MongoClient('localhost', 27017)

In [96]:
PATH = Path('./data')

## Parameters

In [13]:
MOLECULE = 'h2o'

In [14]:
atm = AtmComposition.cliradlw_nongreys()
BANDMOLS = [(mapband_new2old()[band], 'h2o') for band, absorbers in atm.gasinbands.items() if 'h2o' in absorbers]

In [15]:
ATMPROS = [('saw_trpwlayer', 257), ('trp_sawwlayer', 300)]

In [147]:
PARAMS = []
for band, absorber in BANDMOLS:
    for atmpro, tsfc in ATMPROS:
        param = kdist_params(molecule=absorber, band=band)
        param['commitnumber'] = '9dc25c5'
        param['atmpro'], param['tsfc'] = atmpro, tsfc
        param['option_compute_ktable'] = 0
        param['option_compute_btable'] = 0
        PARAMS.append(param)

In [148]:
PARAMS = [LBLnewBestfitParam(**param) for param in PARAMS]

## Cooling rate profiles from all spectral bands in Dataframe
These are also exported to excel files.

In [149]:
bands = list(AtmComposition.cliradlw_nongreys().gasinbands.keys())

In [160]:
a = 'trp_sawwlayer' #'saw_trpwlayer'

with pd.ExcelWriter(PATH/f'h2o_cool_{a}.xlsx') as writer:
    for i, b in enumerate(bands):
        ps = [p for p in PARAMS if p.atmpro==a and mapband_old2new()[p.band]==b]
        datas = [p.modeldata_pymongo(collection=client.lblnew.bestfit_lw) for p in ps]
        assert len(datas) == 1
        data = datas[0]
        df = pd.concat([data.crd_cool.coolrg.sum('g').to_pandas(),
                        data.wgt_cool.coolrg.sum('g').sel(igg=1).to_pandas(),
                        data.wgt_cool.coolrg.sum('g').sel(igg=10).to_pandas()],
                       axis=1, keys=['CRD', 'WGT igg=1', 'WGT igg=10'])
        
        if i==0: bandstot = df
        else: bandstot += df
            
        df.to_excel(writer, sheet_name=f'band {b}')

    bandstot.to_excel(writer, sheet_name=f'bands-tot')

In [162]:
[p.resolve() for p in PATH.iterdir()]

[PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param/sawtrp_swap_atmpros/data/h2o_cool_saw_trpwlayer.xlsx'),
 PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param/sawtrp_swap_atmpros/data/cool_specialprofiles.xlsx'),
 PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param/sawtrp_swap_atmpros/data/h2o_cool_trp_sawwlayer.xlsx')]

# fin